<a href="https://colab.research.google.com/github/Maxasaran/labs/blob/main/SparkLab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkApp') \
                    .getOrCreate()
tiktokData2022 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/content/TikTok_songs_2022.csv")

tiktokData2022.createOrReplaceTempView("TikTok_songs_2022")

In [ ]:


tiktokData2022.createOrReplaceTempView("TikTok_songs_2022")
from pyspark.sql.functions import desc

sqlWay = spark.sql("""
SELECT track_name,artist_name
FROM TikTok_songs_2022
LIMIT 5
""")

dataFrameWay = tiktokData2022\
    .select(
    "track_name",
    "artist_name")\
    .limit(5)

sqlWay.show()
dataFrameWay.show()

tiktokData2022.dtypes

In [ ]:

#1: Получите список исполнителей с популярностью больше 90%
artist_name_sql = spark.sql(""" 
SELECT DISTINCT artist_name 
FROM TikTok_songs_2022 
WHERE artist_pop > 90
""")

artist_name = tiktokData2022.filter("artist_pop > 90") \
                .select("artist_name") \
                .dropDuplicates()

print(artist_name_sql.rdd.map(lambda x: x[0]).collect())
print(artist_name.rdd.map(lambda x: x[0]).collect())


In [ ]:
# 2: Выведите список исполнителей и количество их треков в порядке уменьшения.
# Ограничьте выборку исполнителями, у которых не менее 3 треков в наборе данных.

artist_tracks_sql = spark.sql("""
SELECT artist_name, COUNT(*) as number_of_tracks
FROM TikTok_songs_2022
GROUP BY artist_name 
HAVING COUNT(*) >= 3
ORDER BY number_of_tracks DESC
""")

artist_tracks = tiktokData2022.groupby("artist_name") \
                    .count().withColumnRenamed("count", "number_of_tracks") \
                    .sort(desc("number_of_tracks")) \
                    .filter("number_of_tracks >= 3")
artist_tracks_sql.show()
artist_tracks.show()

In [ ]:
# 3: Выведите список названий треков, средняя громкость которых лежит в диапазоне от -10 до -9 Дб.

# ['Glimpse of Us', 'Left and Right (Charlie Puth) [feat. Jung Kook of BTS] - Sped Up Version',
# 'Frozen', "Can't Get You Out of My Head", 'I Hate U', 'I AM WOMAN', 'Buss It', 'Whatever You Like',
# 'MONEY', 'Freaks', 'telepatía', 'Falling Down', 'Party Girl', 'Skechers', 'Good Looking']
songs_list_sql = spark.sql("""
SELECT track_name
FROM TikTok_songs_2022
WHERE loudness
BETWEEN -10 AND -9
""")
songs_list = tiktokData2022 \
              .filter("loudness > -10") \
              .filter("loudness < -9") \
              .select("track_name")

print(songs_list_sql.rdd.map(lambda x: x[0]).collect())
print(songs_list.rdd.map(lambda x: x[0]).collect())


In [ ]:
#4: Выведите список треков, в названиях которых есть слово "love" (без учета регистра).

# ['Another Love', 'love nwantiti (ah ah ah)', 'Tainted Love', 'Looking for Love', 'Free Falling Love',
# 'Electric Love', 'Feel Your Love', 'What You Know Bout Love', 'Savage Love (Laxed - Siren Beat)',
# 'Love Me Again', 'Lovely']

songs_list_lower_sql = spark.sql("""
SELECT track_name
FROM TikTok_songs_2022
WHERE track_name ILIKE '%love%'
""")

songs_list_lower = tiktokData2022 \
                      .select("track_name") \
                      .where(tiktokData2022.track_name \
                      .ilike("%love%")) 
                      
print(songs_list_lower_sql.rdd.map(lambda x: x[0]).collect())
print(songs_list_lower.rdd.map(lambda x: x[0]).collect())

In [ ]:
#5: Получите таблицу из песен Арианы Гранде и Astelle. Отсортируйте их по примеру ниже.
two_artist_sql = spark.sql("""
SELECT track_name, artist_name
FROM TikTok_songs_2022
WHERE artist_name IN ('Ariana Grande','Astelle')
ORDER BY artist_name, track_name
""")

two_artist = tiktokData2022 \
              .select("track_name", "artist_name") \
              .where(tiktokData2022.artist_name \
              .isin("Ariana Grande","Astelle")) \
              .orderBy("artist_name","track_name") \

two_artist_sql.show()
two_artist.show()


In [ ]:
#6: Какую задачу решает приведенный ниже код?
# В чем суть метода K средних?
# Какие еще существуют методы машинного обучения для решения данной задачи?
# Какие выводы можно сделать о полученных группах?

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.sql import Row
from pyspark.ml.feature import MinMaxScaler
from IPython.display import display
import numpy as np

k = 20 #Количество итераций образования центроида

np.set_printoptions(precision=3, suppress=True)  #задает параметры вывода массива на экран,
                                                 #precision - количество цифр после запятой для чисел с плавающей точкой
                                                 #suppress - если true то числа с плавающей точкой выводятся как числа с фикс точкой

columns_to_scale = ["danceability", "energy", "speechiness", "acousticness",
                    "instrumentalness","liveness", "valence", "duration_ms",
                    "tempo", "track_pop"]  #столбцы, используемые в обучении 

assemblers = [VectorAssembler(inputCols = [col], outputCol = col + "_vec") for col in columns_to_scale]  #получение векторных значений из значений столбцов, используемых в обучении,
                                                                                                         # и их запись в новые столбцы
scalers = [MinMaxScaler(inputCol = col + "_vec", outputCol = col + "_scaled") for col in columns_to_scale] # нормализация значений (диапазон от 0 до 1) векторных значений и запись в новые столбцы

vectorAssembler = VectorAssembler()\
  .setInputCols(["danceability_scaled", "energy_scaled", "speechiness_scaled", "acousticness_scaled",
                 "instrumentalness_scaled", "liveness_scaled", "valence_scaled", "duration_ms_scaled",
                 "tempo_scaled", "track_pop_scaled"])\
  .setOutputCol("features")       #получение результирующей вектор-колонки из нормализованных раннее значений 

transformationPipeline = Pipeline()\
  .setStages(assemblers + scalers + [vectorAssembler])  #устанавливаем методы преобразований в pipeline на случай, 
                                                        #если будут загружаться новые данные,чтобы они проходили те же преобразования

fittedPipeline = transformationPipeline.fit(tiktokData2022) #"подгонка" обучающих данных
transformedDF = fittedPipeline.transform(tiktokData2022)  #возвращение формата выходных данных к исходному (тут если верно понял функцию transfom https://dfedorov.spb.ru/pandas/%D0%9F%D0%BE%D0%BD%D0%B8%D0%BC%D0%B0%D0%BD%D0%B8%D0%B5%20%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8%20transform%20%D0%B2%20Pandas.html)
                                                         

kmeans = KMeans()\
  .setK(k)\
  .setSeed(300)  #создаем экземпляр обучаемой модели

kmModel = kmeans.fit(transformedDF)  #обучение

df_pred = kmModel.transform(transformedDF) #возвращение к исходному формату данных

centers = kmModel.clusterCenters() #получаем список каждого значения, образующего  центроид
for i in range(0, k):
    print("group " + str(i))
    print(centers[i])
    df = df_pred.selectExpr("track_name", "danceability", "energy", "speechiness",
                        "acousticness","instrumentalness","liveness", "valence",
                        "duration_ms", "tempo", "track_pop")\
        .where("prediction = " + str(i))\
        .toPandas()
    display(df)
#Какую задачу решает приведенный ниже код?    
#Отображение результатов обучения - Вывод значений, образующих центроид, в виде списка для каждого столбца
#и отображение в виде таблицы списка песен с соответсвующими им значеними столбцов из датасета, наиболее соответствующими значениям образующим центроид
#Данный код решает задачу поиска групп песен наиболее близких к друг другу по совокупности значений параметров, указанных в переменной columns_to_scale

# В чем суть метода K средних?
#k-means — это алгоритм кластеризации, в котором «k» центров случайным образом назначаются в данных.
#Ближайшие к этой точке точки затем «присваиваются» классу, и вычисляется центр назначенных точек. 
#Эта центральная точка называется центроидом. 
#Затем мы помечаем точки, ближайшие к этому центроиду, и сдвигаем центроид к новому центру этого кластера точек.
#Мы повторяем этот процесс для конечного набора итераций или до сходимости (наши центральные точки перестают меняться).

# Какие еще существуют методы машинного обучения для решения данной задачи?
#k-medoids, агломеративный иерархический метод, дивизимный иерархический метод, плотностной метод (https://scikit-learn.ru/clustering/)

# Какие выводы можно сделать о полученных группах?
#Вывод о группах песен сделать сложно, тк используется большое количество столбцов (параметров песен) и все они обьединены по случайным значениям, образующим центроид,
#то есть обьединение групп можно сказать пседослучайное. Чтобы это, как мне кажется, имело практический смысл нужно уменьшить количество параметров(столбцов) для обучения
# Одно можно сказать точно, инструментальных треков очень мало
